In [1]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint

## Hypothesis
- Backend
- Python & Java
- Junior positions
- 경기도/분당 쪽 포지션들 위주
(사실 뻔하긴해...)

In [2]:
data = json.load(Path("job_positions.json").open('r'))
positions = data["jobPositions"]
df = pd.DataFrame(positions)
# df = df["id title address career tagNames isEnded technicalTags company".split()]
df.head()

,id,title,address,career,tagNames,isBookmarked,isEnded,technicalTags,company,isRecommended
0,26285,[음악플랫폼FLO] Backend 개발자,"대한민국 서울특별시 서초구 강남대로 311, 15층 플로하우스",3 ~ 15년,"[서버/백엔드, Java, Go, SQL, NoSQL, Kotlin, Python,...",True,False,"[{'id': 1182, 'name': 'Java'}, {'id': 1163, 'n...","{'id': 1039, 'name': '드림어스컴퍼니', 'averageRespon...",False
1,26293,데이터 엔지니어(DE),"서울 송파구 올림픽로35다길 42, (신천동)",경력 무관,"[데이터 엔지니어, Python, SQL, Tableau, MySQL, AWS S3...",True,False,"[{'id': 1230, 'name': 'Python'}, {'id': 1251, ...","{'id': 1709, 'name': '에코마케팅', 'averageResponse...",False
2,15930,웹(Web) 백엔드 개발자,"서울 종로구 청계천로 85, (관철동) 10층",경력 무관,[서버/백엔드],True,False,[],"{'id': 9257, 'name': '누아', 'averageResponseTim...",False
3,24124,GAIA 연구원,"경기 성남시 분당구 정자일로 45, (금곡동) 티맥스타워",경력 무관,"[웹 풀스택, 아이폰 앱, 시스템/네트워크, Java, JavaScript, C++]",True,False,"[{'id': 1182, 'name': 'Java'}, {'id': 1183, 'n...","{'id': 556, 'name': '티맥스', 'averageResponseTim...",False
4,25460,[AIMMO] 데이터프로세싱팀,"경기 성남시 분당구 벌말로50번길 41, (야탑동) 투아이센터 7층",경력 무관,"[데이터 엔지니어, OpenCV, GCP(Google Cloud Platform),...",True,False,"[{'id': 1220, 'name': 'OpenCV'}, {'id': 1597, ...","{'id': 2454, 'name': '에이모', 'averageResponseTi...",False


In [3]:
# tags = list(map(lambda x: x.strip(),df["tagNames"].explode().unique()))
tag_df = df["tagNames"].apply(pd.Series)
tag_df["id"] = df["id"]
tag_df = pd.get_dummies(tag_df, prefix="")
tag_df.head()

,id,_개발PM,_데이터 엔지니어,_서버/백엔드,_웹 풀스택,_Django,_Java,_OpenCV,_Python,_시스템/네트워크,...,_Java,_Linux,_Python,_CI/CD,_MongoDB,_Spring,_C,_Jenkins,_Redis,_Apache ZooKeeper
0,26285,False,False,True,False,False,True,False,False,False,...,False,True,False,False,True,False,False,False,True,True
1,26293,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,15930,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,24124,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,25460,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
print(df.career.explode().value_counts())
exp_df = df["career"].apply(pd.Series)
exp_df["id"] = df["id"]
exp_df = pd.get_dummies(exp_df, prefix="")
exp_df.columns = "id junior mid1 mid2 senior dontcare".split()
exp_df.head()

career
경력 무관      8
3 ~ 15년    1
3 ~ 10년    1
0 ~ 4년     1
2 ~ 8년     1
Name: count, dtype: int64


,id,junior,mid1,mid2,senior,dontcare
0,26285,False,False,False,True,False
1,26293,False,False,False,False,True
2,15930,False,False,False,False,True
3,24124,False,False,False,False,True
4,25460,False,False,False,False,True


In [13]:
import requests
job_url = "https://career.programmers.co.kr/api/job_positions/{}"

lat_long = []
rows = []

for job_id in df["id"]:
    write_path = Path()/"jobs"/(str(job_id)+".json")
    if not write_path.exists():
        job_data = requests.get(job_url.format(job_id)).json()
        write_path.parent.mkdir(parents=True, exist_ok=True)
        json.dump(job_data,write_path.open("w", encoding='utf-8-sig'), ensure_ascii=False, indent=4)
    job_data = json.load(write_path.open('r',encoding='utf-8-sig'))
    job_data = job_data["jobPosition"]
    rows.append([
        job_id, job_data["description"]
    ,job_data["additionalInformation"]
    ,job_data["preferredExperience"]
    ,job_data["requirement"]])
    lat_long.append([job_id, job_data["latitude"], job_data["longitude"]])

job_desc_df = pd.DataFrame(rows)
job_desc_df.columns = "id description additionalInfo experience requirement".split()
display(job_desc_df)

latlong_df = pd.DataFrame(lat_long)
latlong_df.columns = "id lat long".split()
# latlong_df = latlong_df.dropna()
display(latlong_df)


,id,description,additionalInfo,experience,requirement
0,26285,* Java Spring framework와 Golang을 사용하여 이벤트 기반 음...,- - -\r\n\r\n**[팀소개]**\r\n음악 도메인에 특화된 메타 데이터 체...,"* 개발 경력 7년 이상인 분\r\n* Java, Kotlin, Golang, Py...",* 개발 경력 최소 3년 이상인 분\r\n* Java spring framework...
1,26293,"* <span class=""colour"" style=""color: rgb(0, 0,...","## **에코마케팅은,**\r\n<span class=""colour"" style=""...","* 유관 경력 6개월 이상 보유하신 분\r\n* <span class=""colour...","* <span class=""colour"" style=""color: rgb(0, 0,..."
2,15930,* 예약 관리 시스템 개발\r\n* 차세대 여행 플랫폼 기술 서비스 연동\r\n* ...,### <혜택 및 복지>\r\n\r\n* 청계천 view의 오피스\r\n* 지하철 ...,"* 항공, 숙박 등 예약 플랫폼 연동 개발 경험\r\n* 새로운 언어 및 개발에 대...","* Java, Spring Framework 개발 경험\r\n* Web 서비스 개발..."
3,24124,### 1\. OF연구본부 WAPL Docs 개발 연구원\r\n<br>\r\n**1...,### 채용 절차\r\n\r\n**서류 전형 → 코딩테스트 → 1차 면접 → 2차 ...,### 1\. OF연구본부 WAPL Docs 개발 연구원\r\n<br>\r\n<sp...,### 1\. OF연구본부 WAPL Docs 개발 연구원\r\n<br>\r\n**이...
4,25460,"* 다양한 데이터 유형(이미지, 동영상, Point Cloud)의 레이블링 데이터(...",**팀 문화**\r\n01\. Accountability\r\n자율성이 더 나은 퍼...,"* AWS, Azure, GCP, NCP 등 클라우드 사용 경험\r\n* 인공지능 ...",* 파이썬 사용 경험\r\n* 반복 작업을 자동화 하고 기술 부채를 개선하고자 노력...
5,25492,* 자율주행 및 ADAS 개발을 위한 인공지능 데이터 생산 외주 용역 프로젝트 운...,**팀 문화**\r\n01\. Accountability\r\n자율성이 더 나은 퍼...,* 스스로를 끊임없이 학습하며 성장하는 분\r\n* 업무 자동화 프로세스를 적용 경...,* AI 학습 데이터 생산 및 관련 프로젝트 운영 경험 3년 이상\r\n* Auto...
6,15219,* 웹 개발 (전자게약 및 영업사원과 대리점에 사용하는 안드로이드/아이폰 용 앱개발...,None,"* 협업,소통, 개발이 우수한자\r\n* 회계적 지식, 통계적 사고 보유자","* 관련 경력 보유자\r\n* php, mysql, html, javascript 가능자"
7,24306,"<h1><img src=""https://opening-attachments.gree...",None,None,None
8,19329,"### <span class=""colour"" style=""color:rgb(40, ...","### <span class=""colour"" style=""color:rgb(40, ...","### <span class=""colour"" style=""color:rgb(40, ...","### <span class=""colour"" style=""color:rgb(40, ..."
9,24125,### 1. 서버 운영체제 및 서버 가상화 서비스 연구원\r\n\r\n**1) 합류...,### 채용 절차\r\n<br>\r\n**서류 전형 → 코딩테스트 → 1차 면접 →...,### 1. 서버 운영체제 및 서버 가상화 서비스 연구원\r\n\r\n\- 하이퍼바...,### 1. 서버 운영체제 및 서버 가상화 서비스 연구원\r\n\r\n\- 4년제 ...


,id,lat,long
0,26285,37.490895,127.030299
1,26293,37.515254,127.103069
2,15930,37.568631,126.987150
3,24124,37.353825,127.104946
4,25460,37.411946,127.143291
5,25492,37.411946,127.143291
6,15219,37.566679,126.912299
7,24306,37.502327,127.044445
8,19329,37.353825,127.104946
9,24125,37.353825,127.104946


In [6]:
from utils import preprocess_text

# TODO: preprocess text with html?
for col in job_desc_df.columns:
    job_desc_df[col] = job_desc_df[col].apply(lambda x: preprocess_text(x) if x else x)
# job_desc_df["additionalInfo"] = job_desc_df["additionalInfo"].apply(preprocess_text)


job_desc_df["description"].to_csv("desc.csv")
job_desc_df["additionalInfo"].to_csv("add.csv")
job_desc_df["experience"].to_csv("exp.csv")
job_desc_df["requirement"].to_csv("req.csv")
job_desc_df

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /home/jhojin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jhojin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,description,additionalInfo,experience,requirement
0,Java Spring framework와 Golang을 사용하여 이벤트 기반 음악 ...,팀소개 음악 도메인에 특화된 메타 데이터 체계와 자동화된 입수 시스템을 활용하여 다...,개발 경력 7년 이상인 분 Java Kotlin Golang Python Rust ...,개발 경력 최소 3년 이상인 분 Java spring framework에 능숙하신 ...
1,span classcolour stylecolor rgb0 0 0현업 부서와의 긴밀...,에코마케팅은 span classcolour stylecolor rgba0 0 0 0...,유관 경력 6개월 보유하신 분 span classcolour stylecolor r...,span classcolour stylecolor rgb0 0 0국내외 4년제 대학...
2,예약 관리 시스템 개발 차세대 여행 플랫폼 기술 서비스 연동 항공권 예약 서비스 개...,혜택 복지 청계천 view의 오피스 지하철 1 2 3 5호선 도보 7분 이내 업무용...,항공 숙박 예약 플랫폼 연동 개발 경험 새로운 언어 개발에 대한 흥미와 도전정신,Java Spring Framework 개발 경험 Web 서비스 개발 경험 논리적 ...
3,1 OF연구본부 WAPL Docs 개발 연구원 br 1 합류하게 될 팀에 알려드려요...,채용 절차 서류 전형 코딩테스트 1차 면접 2차 면접 처우 협의 최종합격 span ...,1 OF연구본부 WAPL Docs 개발 연구원 br span classcolour ...,1 OF연구본부 WAPL Docs 개발 연구원 br 분과 함께하고 싶어요 span ...
4,다양한 데이터 유형이미지 동영상 Point Cloud의 레이블링 데이터Json Fo...,팀 문화 01 Accountability 자율성이 나은 퍼포먼스를 가져온다고 생각합...,AWS Azure GCP NCP 클라우드 사용 경험 인공지능 기술에 대한 관심 대용...,파이썬 사용 경험 반복 작업을 자동화 기술 부채를 개선하고자 노력하시는 분 객체지향...
5,자율주행 ADAS 개발을 위한 인공지능 데이터 생산 외주 용역 프로젝트 운영 관리 ...,팀 문화 01 Accountability 자율성이 나은 퍼포먼스를 가져온다고 생각합...,스스로를 끊임없이 학습하며 성장하는 분 업무 자동화 프로세스를 적용 경험 이공계 전...,AI 학습 데이터 생산 관련 프로젝트 운영 경험 3년 Automotive 기업에서 ...
6,웹 개발 전자게약 영업사원과 대리점에 사용하는 안드로이드아이폰 용 앱개발,None,협업소통 개발이 우수한자 회계적 지식 통계적 사고 보유자,관련 경력 보유자 php mysql html javascript 가능자
7,h1img srchttpsopeningattachmentsgreetinghrcom2...,None,None,None
8,span classcolour stylecolorrgb40 42 49 합류하게 될 ...,span classcolour stylecolorrgb40 42 49 티맥스그룹 연...,span classcolour stylecolorrgb40 42 49 분을 우대해요...,span classcolour stylecolorrgb40 42 49 분과 함께하고...
9,1 서버 운영체제 서버 가상화 서비스 연구원 1 합류하게 될 팀에 알려드려요 Kub...,채용 절차 br 서류 전형 코딩테스트 1차 면접 2차 면접 처우 협의 최종합격 전형...,1 서버 운영체제 서버 가상화 서비스 연구원 하이퍼바이저를 직 간접적으로 다뤄본 경...,1 서버 운영체제 서버 가상화 서비스 연구원 4년제 정규 대학 졸업자 졸업예정자로 ...


## Metrics/Vis
- Positions count by frontend/backend/data, py/java
- Distribution of tags
- Word cloud of JD texts by column, by py/java
- latitude, longitude visualization

- all job postings -> by location -> by tech stack -> ... //https://sankeymatic.com/